# Finite Element Tutorial - Validation with Analytical Solution

This tutorial demonstrates the validation of the finite element analysis results with analytical results for a simple system.

Consider a beam with length L, modulus E, and second moment of area I. Let it be fixed at both ends, and apply a uniform applied load q.

INCLUDE IMAGE OF BEAM

The analytical solution for the deflection in the middle of the beam is then given by

$\delta_{max} = \frac{q x^2}{24 E I} \left(L - x^2\right)$

(Image and formula from https://structx.com/Beam_Formulas_015.html, retrieved 4/25)

## Dependency Importing





In [2]:
import os
import warnings
warnings.simplefilter("always")
from finiteelementanalysis import pre_process as pre
from finiteelementanalysis import pre_process_demo_helper_fcns as pre_demo
from finiteelementanalysis.solver import hyperelastic_solver
from finiteelementanalysis import visualize as viz
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

## Geometry and Mesh Setup

We model only half the beam, for computational efficiency, leaving the end representing the middle free to move vertically (but fixed horizontally). A fine mesh with matching length scales in the x and y directions should provide accurate results.

In [3]:
# for saving files later
tutorials_dir = os.path.abspath('')

# FEA problem info
ele_type = "D2_nn6_tri"
ndof = 2

# Define domain
L = 10.0      # twice length in x-direction
H = 0.2      # height in y-direction
nx = 100       # number of elements in x
ny = 2       # number of elements in y, keep this an even number if you want the analytical solution to be able to compute midline deformation

b = 4 # beam thickness in z-direction (Note - does not change simulation)
I = b*H**3/12


# Generate mesh
coords, connect = pre.generate_rect_mesh_2d(ele_type, 0, -H/2, L/2, H/2, nx, ny)

mesh_img_fname = tutorials_dir + "/a-mesh.png"
pre_demo.plot_mesh_2D(str(mesh_img_fname), ele_type, coords, connect)


![](a-mesh.png "Beam Mesh")

## Specify a Material and the Boundary Conditions

To limit only the horizontal motion of the end of the beam representing the middle, we specify an x-velocity of 0 and a y-velocity of None.

In [4]:
# Choose material properties
mu = 134.6
K = 83.33
#mu = E / (2*(1 + v))
#K = E / (3*(1 - 2*v))
v = (3*mu/K - 2) / (2 + 6*mu/K)
E = mu * 2*(1 + v)
material_props = np.array([mu, K])  # [mu, K]

# Identify boundaries
boundary_nodes, boundary_edges = pre.identify_rect_boundaries(coords, connect, ele_type, 0, L/2, -H/2, H/2)

# Apply boundary conditions:
# 1. Fix left end boundary
fixed_left = pre.assign_fixed_nodes_rect(boundary_nodes, "left", 0, 0)
# 2. Fix right end boundary horizontally (but not vertically)
fixed_right = pre.assign_fixed_nodes_rect(boundary_nodes, "right", 0, None)
# Combine BCs (assuming the functions return arrays of shape (3, n_bc))
fixed_nodes = np.hstack((fixed_left, fixed_right))

# Apply the uniform distributed load
q = -0.02
dload_info = pre.assign_uniform_load_rect(boundary_edges, "top", 0.0, q)

## Run the Solver

In [5]:
# Number of incremental loading steps
nr_num_steps = 100

# Run the solver
displacements_all, nr_info_all = hyperelastic_solver(
    material_props,
    ele_type,
    coords.T,      # solver expects coords as (ncoord, n_nodes)
    connect.T,     # and connectivity as (n_nodes_per_elem, n_elems)
    fixed_nodes,
    dload_info,
    nr_print=True,
    nr_num_steps=nr_num_steps,
    nr_tol=1e-8,
    nr_maxit=30,
)

final_disp = displacements_all[-1]  # final global displacement vector (length = n_nodes * ndof)

# Analytical solution: For a homogeneous extension,
#   u_x(x) = (lambda_target - 1) * x, and u_y(x) = 0.
# Extract nodes near mid-height to get a 1D slice.
tol_y = H / 20.0  # tolerance for y coordinate
mid_nodes = [i for i in range(coords.shape[0]) if abs(coords[i, 1] - H/2) < tol_y]
mid_nodes = sorted(mid_nodes, key=lambda i: coords[i, 0])  # sort by x-coordinate

# Extract y-coordinates and computed u_y from the final displacement.
x_vals = np.array([coords[i, 0] for i in mid_nodes])
computed_u_y = np.array([final_disp[ndof * i + 1] for i in mid_nodes])

Step 0, load factor = 0.010
Iteration 1, Correction=1.000000e+00, Residual=3.705457e-08, tolerance=1.000000e-08
Iteration 2, Correction=1.777474e-02, Residual=2.039445e-06, tolerance=1.000000e-08
Iteration 3, Correction=6.466182e-04, Residual=6.788810e-10, tolerance=1.000000e-08
Iteration 4, Correction=3.177377e-08, Residual=9.385799e-13, tolerance=1.000000e-08
Iteration 5, Correction=4.179202e-15, Residual=2.678291e-17, tolerance=1.000000e-08
Step 1, load factor = 0.020
Iteration 1, Correction=4.910756e-01, Residual=3.705457e-08, tolerance=1.000000e-08
Iteration 2, Correction=2.858103e-02, Residual=1.824693e-06, tolerance=1.000000e-08
Iteration 3, Correction=9.399079e-04, Residual=6.349113e-09, tolerance=1.000000e-08
Iteration 4, Correction=2.549357e-07, Residual=7.213237e-12, tolerance=1.000000e-08
Iteration 5, Correction=4.090293e-14, Residual=3.313346e-17, tolerance=1.000000e-08
Step 2, load factor = 0.030
Iteration 1, Correction=3.075935e-01, Residual=3.705457e-08, tolerance=1.000

## Plot the Results

In [6]:
# Analytical solution: u_y(x)
analytical_u_y = q*x_vals**2/(24*E*I) * (L-x_vals)**2

# Plot the computed and analytical u_y vs. x.
plt.figure(figsize=(8, 6))
plt.plot(x_vals, computed_u_y, 'ro-', label="Computed u_y")
plt.plot(x_vals, analytical_u_y, 'b--', label="Analytical u_y")
plt.xlabel("x (m)")
plt.ylabel("u_y (m)")
plt.title("Comparison of u_y(x): Computed vs. Analytical")
plt.legend()
plt.grid(True)
plt.tight_layout()

# Save the plot image
img_fname = tutorials_dir + "/a-u_y_comparison.png"
plt.savefig(str(img_fname))

# Plot the percent error
plt.figure(figsize=(8, 6))
plt.plot(x_vals, (computed_u_y - analytical_u_y) / analytical_u_y, 'ro-', label="Percent Error")
plt.xlabel("x (m)")
plt.ylabel("Percent Error")
plt.title("Deflection Percent Error")
plt.legend()
plt.grid(True)
plt.tight_layout()

# Save the plot image
img_fname = tutorials_dir + "/a-u_y_percenterror.png"
plt.savefig(str(img_fname))

# Save an animation of the deformation
img_name = tutorials_dir + "/a-deformation.gif"
gif_displacements = displacements_all
for i_extend in range(20): gif_displacements += [final_disp]
viz.make_deformation_gif(gif_displacements, coords, connect, ele_type, img_name, interval=40)

/scratch/337191.1.ood/ipykernel_2345549/595809290.py:21: RuntimeWarning: invalid value encountered in divide
  plt.plot(x_vals, (computed_u_y - analytical_u_y) / analytical_u_y, 'ro-', label="Percent Error")


![](a-u_y_comparison.png "Analytic vs Computed Deflection")

![](a-u_y_percenterror.png "Deflection Percent Error")

The results are correlated, but seem to differ non-negligibly. The hyperelastic solver seems to yield different results than the elastic analytical model, even in this simple case. For the analytical model, a thickness in the z-direction had to be specified to compute a second moment of area, while the solver did not take any z-direction thickness into account.

![](a-deformation.gif "Animation of the Beam Deformation")